In [9]:
!pip install pillow numpy opencv-python matplotlib pinecone rembg onnxruntime onnxruntime-gpu tensorflow

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 3.1/390.3 MB 15.3 MB/s eta 0:00:26
    --------------------------------------- 6.0/390.3 MB 15.4 MB/s eta 0:00:25
    --------------------------------------- 9.2/390.3 MB 15.0 MB/s eta 0:00:26
   - -------------------------------------- 12.3/390.3 MB 15.4 MB/s eta 0:00:25
   - -------------------------------------- 15.7/390.3 MB 15.5 MB/s eta 0:00:25
   - -------------------------------------- 18.6/390.3 MB 15.4 MB/s eta 0:00:25
   -- ------------------------------------- 21.8/390.3 MB 15.5 MB/s eta 0:00:24
   -- ------------------------------------- 24.9/390.3 MB 15.5 MB/s eta 0:00:24
   -- ------------------------------------- 28.0/390.3 MB 15.5 MB/s eta 0:00:24
   --- ------------------------------------ 31.2/390.3 MB 15.5 MB/s eta 0:00:24
   --- ------------------------------------ 34.1/390.3 MB 1

In [2]:
import requests
from PIL import Image
import numpy as np
import cv2
import os
from pinecone import Pinecone
import matplotlib.pyplot as plt

In [7]:
from pathlib import Path
from rembg import remove, new_session
from PIL import Image
import io

# Initialize Rembg session
session = new_session()

# Define directories
input_dir = Path('../data/0_raw/')
output_dir = Path('../data/1_staging/')
output_dir.mkdir(parents=True, exist_ok=True)

for file in input_dir.glob('*.jpg'):
    input_path = str(file)
    output_path = str(output_dir / (file.stem + '.out.jpg'))
    
    with open(input_path, 'rb') as i:
        input_content = i.read()
        output_content = remove(input_content, session=session)   
        output_image = Image.open(io.BytesIO(output_content)) # Convert removed background image to Image format for resizing
        if output_image.mode == 'RGBA':
            output_image = output_image.convert('RGB')
        output_image = output_image.resize((224, 224), Image.LANCZOS)# Resize the image for vgg16 use later
        output_image.save(output_path, format='JPEG')

    print(f"Processed and saved: {output_path}")

Processed and saved: ..\data\1_staging\001_08194468.out.jpg
Processed and saved: ..\data\1_staging\001_08212dcd.out.jpg
Processed and saved: ..\data\1_staging\001_21a7d5e6.out.jpg
Processed and saved: ..\data\1_staging\001_2288a4f6.out.jpg
Processed and saved: ..\data\1_staging\001_504d320d.out.jpg
Processed and saved: ..\data\1_staging\001_5992faf7.out.jpg
Processed and saved: ..\data\1_staging\001_5ef3e95c.out.jpg
Processed and saved: ..\data\1_staging\001_986d6c22.out.jpg
Processed and saved: ..\data\1_staging\001_9adc92c2.out.jpg
Processed and saved: ..\data\1_staging\001_9cd1160a.out.jpg
Processed and saved: ..\data\1_staging\001_a51bb26a.out.jpg
Processed and saved: ..\data\1_staging\001_beebcee2.out.jpg
Processed and saved: ..\data\1_staging\001_c04300ef.out.jpg
Processed and saved: ..\data\1_staging\001_cb004eea.out.jpg
Processed and saved: ..\data\1_staging\001_d3323f3c.out.jpg
Processed and saved: ..\data\1_staging\001_dfb62d96.out.jpg
Processed and saved: ..\data\1_staging\0

In [18]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image

# Initialize Pinecone
pinecone = Pinecone(api_key='pcsk_6mfAsZ_7xBhAxC4ieaUqRR6S3josinEyvg37siV71awC2gXXPdk7Ycnda9h8HkB5ZAPp5N')
index = pc.Index('datalakes-ing3')

In [19]:
base_model = VGG16(weights='imagenet', include_top=True)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output) # Load VGG16 excluding top layers

def extract_features(img_path):
    
    img = image.load_img(img_path, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    
    vgg16_feature = model.predict(img_data)
    flat_feature = vgg16_feature.flatten()
    return flat_feature

In [20]:
for filename in os.listdir('../data/1_staging/'):
    img_path = os.path.join('../data/1_staging/', filename)
    try:
        features = extract_features(img_path)
        features /= np.linalg.norm(features) # Normalize for cosine similarity to work
        index.upsert([(filename, features)]) # Index the feature vector in Pinecone
        #print(filename, features)
    
    except Exception as e:
        print(f"Error processing {filename}: {e}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [23]:
status = index.describe_index_stats()
print(status)

{'dimension': 4096,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 1869}},
 'total_vector_count': 1869,
 'vector_type': 'dense'}
